# Criação do modelo de classificação para o chatbot

## Importando os módulos necessários:

In [ ]:
!pip install tensorflow
!pip install --upgrade tensorflow_text
!pip install --upgrade tensorflow_hub
!pip install pandas
!pip install tflearn
!pip install sklearn
!pip install numpy

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from tensorflow import keras

## Usando uma build pré-treinada e iniciando modelo

+ Usamos um modelo pré-treinado do Tensorflow Hub
+ Usamos o pré-processador equivalente ao modelo escolhido
+ Durante a execução, alguns warnings são exibidos. Isto se dá pela falta de uma GPU na máquina, então o aviso pode ser ignorado.

In [ ]:
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)


## Importando os dados para treinamento

+ Os dados devem estar armazenados no mesmo diretório que o código
+ Serão armazenados na forma de Dataframe, usando Pandas
+ Após importação, é feita a transformação da variável categórica "Tag"

In [ ]:
df = pd.read_csv("data.csv")

df['funny'] = df['Tag'].apply(lambda x: 1 if x=='funny' else 0)
df['goodbye'] = df['Tag'].apply(lambda x: 1 if x=='goodbye' else 0)
df['greeting'] = df['Tag'].apply(lambda x: 1 if x=='greeting' else 0)
df['me'] = df['Tag'].apply(lambda x: 1 if x=='me' else 0)
df['query'] = df['Tag'].apply(lambda x: 1 if x=='query' else 0)
df['thanks'] = df['Tag'].apply(lambda x: 1 if x=='thanks' else 0)
df['use'] = df['Tag'].apply(lambda x: 1 if x=='use' else 0)

df.head(1)

## Criação de dados de treinamento

Nesta etapa, os dados salvos no dataframe são separados em duas partes:

+ Uma lista de frases, seriam os dados de entrada
+ Uma lista contendo a categoria (transformada em uma lista) equivalente a cada frase.

Cada categoria é representada por uma lista de zeros contendo apenas um número um.

In [ ]:
list = df.values.tolist()
intents = []
sents = []
for i in range(len(list)):
    line = list[i][2:]
    intents.append(line)
    sents.append(list[i][0])

## Criando o modelo

+ A camada de input receberá um texto e passará adiante este texto pré-processado pelo modelo retirado do Tensorflow Hub
+ Este texto pré-processado é processado pelas camadas do bert, modelo extraído do Tensorflow Hub
+ Para evitar overfitting, foi adicionada uma camada de dropout
+ Após o dropout, foi feita uma camada de classificação. Ela possui 6 neurônios, pois o sistema possui 6 classificações distintas. A função de ativação escolhida foi a softmax pois esta se sai melhor para tarefas de classificação com múltiplas categorias.
+ O modelo é então criado com a arquitetura definida.

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(7, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])


## Definidas as métricas do modelo e feita a sua compilação

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

## O modelo é então treinado

+ Usa-se como conjunto de treinamento as frases retiradas do dataset
+ Como trata-se de uma operação de aprendizagem supervisionada, as etiquetas para cada entrada são dadas pela categoria transformada anteriormente.
+ usamos 1000 épocas e um tamanho de batch de 10.

In [ ]:
hist = model.fit(sents,intents,epochs=1000, batch_size=10)


## O modelo é então salvo

+ Foi utilizado o modelo no formato hdf5

In [ ]:
model.save('chatModel.h5')